In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
mask = 8  #choose a mask
filename = "detector_responses_ch_bottom.csv"
# filename = "ordered.csv"
df = pd.read_csv(filename)
df.sort_values(by=['mask', 'module', 'LED int'], ascending=[True, True, True], inplace=True) 
df.to_csv(filename, index=False)
df_mask = df[df['mask'] == mask].copy()
blocks = [g.reset_index(drop=True) for _, g in df_mask.groupby("module", sort=False)]


In [ ]:
ncols = 2
nrows = int(np.ceil(len(blocks) / ncols))

fig, axes = plt.subplots(
    nrows, ncols,
    sharex=False, 
    sharey=False,  
    figsize=(6*ncols, 4*nrows)
)

axes = axes.flatten()  

fig.suptitle(f"MASK {mask}", fontsize=16)

for j in range(len(blocks), len(axes)):
    fig.delaxes(axes[j])

for i, block in enumerate(blocks):
    x = block["LED int"].to_numpy()
    y = block["value"].to_numpy()
    detname = block["module"].iloc[0]

    m, q = np.polyfit(x, y, 1)
    one = (-q+1.5)/m
    two = (-q+2)/m
    print(f"{one}, {two}")

    ax = axes[i]
    ax.set_xlabel("LED intensity")
    ax.set_ylabel("Average number of PE [PE]")
    ax.scatter(x, y, color="black", label="Data")
    ax.plot(x, m*x + q, color="red", label=f"Fit: y={m:.4f}x+{q:.2f}")
    ax.set_title(detname)
    ax.legend()

plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.savefig(f"Ch_mask{mask}.png", dpi=300)

plt.show()
